In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Gemma Finetuning (PEFT + TGI)

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_pytorch_gemma_peft_finetuning_hf.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_gemma_peft_finetuning_hf.ipynb">
      <img alt="GitHub logo" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates finetuning and deploying Gemma models with [Vertex AI Custom Training Job](https://cloud.google.com/vertex-ai/docs/training/create-custom-job). Using Vertex AI Pipelines is the quickest way to start finetuning Gemma models, while using a Vertex AI Custom Training Job allows for a higher level of customization and control over the finetuning job. All of the examples in this notebook use parameter efficient finetuning methods [PEFT](https://github.com/huggingface/peft) to reduce training and storage costs.

This notebook deploys the model with the [Text Generation Inference](https://github.com/huggingface/text-generation-inference) docker and uses [Text moderation APIs](https://cloud.google.com/natural-language/docs/moderating-text) to analyze predictions against a list of safety attributes.


### Objective

- Finetune and deploy Gemma models with a Vertex AI Custom Training Job.
- Send prediction requests to your finetuned Gemma model.


### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Cloud NL APIs

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), [Cloud NL API pricing](https://cloud.google.com/natural-language/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

In [ ]:
# @title Setup Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown **[Optional]** Set the GCS BUCKET_URI to store the experiment artifacts, if you want to use your own bucket. **If not set, a unique GCS bucket will be created automatically on your behalf**.

import json
import os
import sys
import uuid
from datetime import datetime
from typing import Tuple

from google.cloud import aiplatform, language

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, please change the value yourself below.
now = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_URI = "gs://"  # @param {type:"string"}
if not BUCKET_URI.strip() or BUCKET_URI == "gs://":
    # Create a unique GCS bucket for this notebook if not specified
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}-{str(uuid.uuid4())[:4]}"
    ! gsutil mb -l {REGION} {BUCKET_URI}
assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."

# Gets the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print("Using this default Service Account:", SERVICE_ACCOUNT)
print(f"Using this GCS Bucket: {BUCKET_URI}")

# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Enable Vertex AI, Cloud Compute, and Cloud Language APIs.
! gcloud config set project $PROJECT_ID
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com language.googleapis.com

# @markdown ## Access Gemma Models

# @markdown Please provide a Hugging Face User Access Token (read) to access the Gemma models. You can follow the [Hugging Face documentation](https://huggingface.co/docs/hub/en/security-tokens) to create a **read** access token and put it in the `HF_TOKEN` field below.
HF_TOKEN = ""  # @param {type:"string", isTemplate:true}
assert HF_TOKEN, "Please provide a read HF_TOKEN to load models from Hugging Face."


# The pre-built training and serving docker images.
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-peft-train:20240220_0936_RC01"
TGI_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-hf-tgi-serve:20240220_0936_RC01"

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id=PROJECT_ID)


def get_job_name_with_datetime(prefix: str) -> str:
    """Gets the job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def moderate_text(text: str) -> language.ModerateTextResponse:
    """Calls Vertex AI APIs to analyze text moderations."""
    client = language.LanguageServiceClient()
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    return client.moderate_text(document=document)


def show_text_moderation(text: str, response: language.ModerateTextResponse) -> None:
    """Shows text moderation results."""
    import pandas as pd

    def confidence(category: language.ClassificationCategory) -> float:
        return category.confidence

    columns = ["category", "confidence"]
    categories = sorted(response.moderation_categories, key=confidence, reverse=True)
    data = ((category.name, category.confidence) for category in categories)
    df = pd.DataFrame(columns=columns, data=data)

    print(f"Text analyzed:\n{text}")
    print(df.to_markdown(index=False, tablefmt="presto", floatfmt=".0%"))


def deploy_model_tgi(
    model_name: str,
    model_id: str,
    service_account: str,
    machine_type: str = "g2-standard-8",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    max_input_length: int = 512,
    max_total_tokens: int = 2048,
    max_batch_prefill_tokens: int = 2048,
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys models with TGI on GPU in Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")

    env_vars = {
        "MODEL_ID": model_id,
        "NUM_SHARD": f"{accelerator_count}",
        "MAX_INPUT_LENGTH": f"{max_input_length}",
        "MAX_TOTAL_TOKENS": f"{max_total_tokens}",
        "MAX_BATCH_PREFILL_TOKENS": f"{max_batch_prefill_tokens}",
        "DEPLOY_SOURCE": "notebook",
    }

    if HF_TOKEN:
        env_vars["HUGGING_FACE_HUB_TOKEN"] = HF_TOKEN

    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=TGI_DOCKER_URI,
        serving_container_ports=[80],
        serving_container_environment_variables=env_vars,
        serving_container_shared_memory_size_mb=(16 * 1024),  # 16 GB
    )

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    return model, endpoint


def get_quota(project_id: str, region: str, resource_id: str) -> int:
  """Returns the quota for a resource in a region. Returns -1 if can not figure out the quota."""
  service_endpoint = "aiplatform.googleapis.com"
  quota_list_output = !gcloud alpha services quota list --service=$service_endpoint  --consumer=projects/$project_id --filter="$service_endpoint/$resource_id" --format=json
  # Use '.s' on the command output because it is an SList type.
  quota_data = json.loads(quota_list_output.s)
  if len(quota_data) == 0 or "consumerQuotaLimits" not in quota_data[0]:
    return -1
  if len(quota_data[0]["consumerQuotaLimits"]) == 0 or "quotaBuckets" not in quota_data[0]["consumerQuotaLimits"][0]:
    return -1
  all_regions_data = quota_data[0]["consumerQuotaLimits"][0]["quotaBuckets"]
  for region_data in all_regions_data:
    if region_data.get('dimensions') and region_data['dimensions']['region'] == region:
      if 'effectiveLimit' in region_data:
        return int(region_data['effectiveLimit'])
      else:
        return 0
  return -1


def get_resource_id(accelerator_type: str, is_for_training: bool) -> str:
  """Returns the resource id for a given accelerator type and the use case.
  Args:
    accelerator_type: The accelerator type.
    is_for_training: Whether the resource is used for training. Set false
    for serving use case.
  Returns:
    The resource id.
  """
  training_accelerator_map = {
      "NVIDIA_TESLA_V100": "custom_model_training_nvidia_v100_gpus",
      "NVIDIA_L4": "custom_model_training_nvidia_l4_gpus",
      "NVIDIA_TESLA_A100": "custom_model_training_nvidia_a100_gpus",
      "NVIDIA_TESLA_T4": "custom_model_training_nvidia_t4_gpus",
      "TPU_V5e": "custom_model_training_tpu_v5e",
      "TPU_V3": "custom_model_training_tpu_v3",
  }
  serving_accelerator_map = {
      "NVIDIA_TESLA_V100": "custom_model_serving_nvidia_v100_gpus",
      "NVIDIA_L4": "custom_model_serving_nvidia_l4_gpus",
      "NVIDIA_TESLA_A100": "custom_model_serving_nvidia_a100_gpus",
      "NVIDIA_TESLA_T4": "custom_model_serving_nvidia_t4_gpus",
      "TPU_V5e": "custom_model_serving_tpu_v5e",
  }
  if is_for_training:
    if accelerator_type in training_accelerator_map:
      return training_accelerator_map[accelerator_type]
    else:
      raise ValueError(
          f"Could not find accelerator type: {accelerator_type} for training."
      )
  else:
    if accelerator_type in serving_accelerator_map:
      return serving_accelerator_map[accelerator_type]
    else:
      raise ValueError(
          f"Could not find accelerator type: {accelerator_type} for serving."
      )


def check_quota(project_id:str, region: str, accelerator_type: str,
                accelerator_count: int, is_for_training: bool):
  """Checks if the project and the region has the required quota."""
  resource_id = get_resource_id(accelerator_type, is_for_training)
  quota = get_quota(project_id, region, resource_id)
  quota_request_instruction = ("Either use "
            "a different region or request additional quota. Follow "
            "instructions here "
            "https://cloud.google.com/docs/quotas/view-manage#requesting_higher_quota"
            " to check quota in a region or request additional quota for "
            "your project.")
  if quota == -1:
    raise ValueError(
            f"""Quota not found for: {resource_id} in {region}.
            {quota_request_instruction}"""
        )
  if quota < accelerator_count:
    raise ValueError(
            f"""Quota not enough for {resource_id} in {region}:
            {quota} < {accelerator_count}.
            {quota_request_instruction}"""
        )

## Finetune with Vertex AI Custom Training Jobs

This section demonstrates how to finetune and deploy Gemma models with PEFT LoRA on Vertex AI Custom Training Jobs. LoRA (Low-Rank Adaptation) is one approach of PEFT (Parameter Efficient FineTuning), where pretrained model weights are frozen and rank decomposition matrices representing the change in model weights are trained during finetuning. Read more about LoRA in the following publication: [Hu, E.J., Shen, Y., Wallis, P., Allen-Zhu, Z., Li, Y., Wang, S., Wang, L. and Chen, W., 2021. Lora: Low-rank adaptation of large language models. *arXiv preprint arXiv:2106.09685*](https://arxiv.org/abs/2106.09685).

In [ ]:
# @title Set Dataset

# @markdown Use the Vertex AI SDK to create and run the custom training jobs.

# @markdown This notebook uses [timdettmers/openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco) dataset as an example.
# @markdown You can set `dataset_name` to any existing [Hugging Face dataset](https://huggingface.co/datasets) name, and set `instruct_column_in_dataset` to the name of the dataset column containing training data. The [timdettmers/openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco) has only one column `text`, and therefore we set `instruct_column_in_dataset` to `text` in this notebook.

# @markdown ### (Optional) Prepare a custom JSONL dataset for finetuning

# @markdown You can prepare a JSONL file where each line is a valid JSON string as your custom training dataset. For example, here is one line from the [timdettmers/openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco) dataset:
# @markdown ```
# @markdown {"text": "### Human: Hola### Assistant: \u00a1Hola! \u00bfEn qu\u00e9 puedo ayudarte hoy?"}
# @markdown ```

# @markdown The JSON object has a key `text`, which should match `instruct_column_in_dataset`; The value should be one training data point, i.e. a string. After you prepared your JSONL file, you can either upload it to [Hugging Face datasets](https://huggingface.co/datasets) or [Google Cloud Storage](https://cloud.google.com/storage).

# @markdown - To upload a JSONL dataset to [Hugging Face datasets](https://huggingface.co/datasets), follow the instructions on [Uploading Datasets](https://huggingface.co/docs/hub/en/datasets-adding). Then, set `dataset_name` to the name of your newly created dataset on Hugging Face.

# @markdown - To upload a JSONL dataset to [Google Cloud Storage](https://cloud.google.com/storage), follow the instructions on [Upload objects from a filesystem](https://cloud.google.com/storage/docs/uploading-objects). Then, set `dataset_name` to the `gs://` URI to your JSONL file. For example: `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl`.

# @markdown Optionally update the `instruct_column_in_dataset` field below if your JSON objects use a key other than the default `text`.

# @markdown ### (Optional) Format your data with custom JSON template

# @markdown Sometimes, your dataset might have multiple text columns and you want to construct the training data with a template. You can prepare a JSON template in the following format:

# @markdown ```
# @markdown {
# @markdown   "description": "A short template for vertex sample dataset.",
# @markdown   "prompt_input": "{input_text}{output_text}",
# @markdown   "prompt_no_input": "{input_text}{output_text}"
# @markdown }
# @markdown ```

# @markdown As an example, the template above can be used to format the following training data (this line comes from `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl`):

# @markdown ```
# @markdown {"input_text":"TRANSCRIPT: \nREASON FOR EVALUATION:,\n\n LABEL:","output_text":"Chiropractic"}
# @markdown ```

# @markdown This example template simply concatenates `input_text` with `output_text`. You can set `template` to `vertex_sample` to try out this built-in template with the dataset `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl`, or build more complicated JSON templates such as [the alpaca example](https://github.com/tloen/alpaca-lora/blob/main/templates/alpaca.json). To use your own JSON template, please [upload it to Google Cloud Storage](https://cloud.google.com/storage/docs/uploading-objects) and put the `gs://` URI in the `template` field below. Leave `instruct_column_in_dataset` as `text`.


# Hugging Face dataset name or gs:// URI to a custom JSONL dataset.
dataset_name = "timdettmers/openassistant-guanaco"  # @param {type:"string"}

# Name of the dataset column containing training text input.
instruct_column_in_dataset = "text"  # @param {type:"string"}

# Optional. Template name or gs:// URI to a custom template.
template = ""  # @param {type:"string"}

Execute the next cell to run the training job.

In [ ]:
# @title Finetune
# @markdown Use the Vertex AI SDK to create and run the custom training jobs. It takes around 40 minutes to finetune Gemma-2B for 1000 steps on 1 NVIDIA_TESLA_V100.

# @markdown **Note**: To finetune the Gemma 7B models, we recommend setting `finetuning_precision_mode` to `4bit` and using NVIDIA_L4 instead of NVIDIA_TESLA_V100.
# @markdown Please click "Show Code" to see more details.

# The Gemma base model.
MODEL_ID = "google/gemma-1.1-2b-it"  # @param["google/gemma-2b", "google/gemma-2b-it", "google/gemma-7b", "google/gemma-7b-it", "google/gemma-1.1-2b-it", "google/gemma-1.1-7b-it"] {isTemplate:true}
# The accelerator to use.
ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"  # @param["NVIDIA_TESLA_V100", "NVIDIA_L4", "NVIDIA_TESLA_A100"]

# Batch size for finetuning.
per_device_train_batch_size = 1  # @param{type:"integer"}
# Runs 10 training steps as a minimal example.
max_steps = 1000  # @param {type:"integer"}
# Precision mode for finetuning.
finetuning_precision_mode = "float16"  # @param["4bit", "8bit", "float16"]
# Learning rate.
learning_rate = 2e-4  # @param{type:"number"}
# LoRA parameters.
lora_rank = 16  # @param{type:"integer"}
lora_alpha = 64  # @param{type:"integer"}
lora_dropout = 0.1  # @param{type:"number"}

# Worker pool spec.
# Gemma only supports single-GPU finetuning.
accelerator_count = 1
machine_type = None
if "7b" in MODEL_ID:
    assert (
        ACCELERATOR_TYPE != "NVIDIA_TESLA_V100"
    ), "Finetuning Gemma 7B models is not supported on NVIDIA_TESLA_V100. Try a GPU with more VRAM."
    if ACCELERATOR_TYPE == "NVIDIA_L4":
        assert (
            finetuning_precision_mode == "4bit"
        ), f"{finetuning_precision_mode} finetuning of Gemma 7B models is not supported on NVIDIA_L4. Try a GPU with more VRAM or use a different precision mode."

if ACCELERATOR_TYPE == "NVIDIA_TESLA_V100":
    machine_type = "n1-standard-8"
elif ACCELERATOR_TYPE == "NVIDIA_L4":
    machine_type = "g2-standard-12"
elif ACCELERATOR_TYPE == "NVIDIA_TESLA_A100":
    machine_type = "a2-highgpu-1g"

assert (
    machine_type
), f"Cannot automatically determine machine type from {ACCELERATOR_TYPE}."

replica_count = 1

check_quota(project_id=PROJECT_ID,
            region=REGION,
            accelerator_type=accelerator_type,
            accelerator_count=accelerator_count,
            is_for_training=True)

# Setup training job.
job_name = get_job_name_with_datetime("gemma-lora-train")

# Pass training arguments and launch job.
train_job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name,
    container_uri=TRAIN_DOCKER_URI,
)

# Create a GCS folder to store the LORA adapter.
lora_adapter_dir = get_job_name_with_datetime("gemma-lora-adapter")
lora_output_dir = os.path.join(STAGING_BUCKET, lora_adapter_dir)

# Create a GCS folder to store the merged model with the base model and the
# finetuned LORA adapter.
merged_model_dir = get_job_name_with_datetime("gemma-merged-model")
merged_model_output_dir = os.path.join(STAGING_BUCKET, merged_model_dir)

train_job.run(
    args=[
        "--task=instruct-lora",
        f"--pretrained_model_id={MODEL_ID}",
        f"--dataset_name={dataset_name}",
        f"--instruct_column_in_dataset={instruct_column_in_dataset}",
        f"--output_dir={lora_output_dir}",
        f"--merge_base_and_lora_output_dir={merged_model_output_dir}",
        f"--per_device_train_batch_size={per_device_train_batch_size}",
        f"--lora_rank={lora_rank}",
        f"--lora_alpha={lora_alpha}",
        f"--lora_dropout={lora_dropout}",
        f"--max_steps={max_steps}",
        "--max_seq_length=512",
        f"--learning_rate={learning_rate}",
        f"--precision_mode={finetuning_precision_mode}",
        f"--template={template}",
        f"--huggingface_access_token={HF_TOKEN}",
    ],
    environment_variables={"WANDB_DISABLED": True},
    replica_count=replica_count,
    machine_type=machine_type,
    accelerator_type=ACCELERATOR_TYPE,
    accelerator_count=accelerator_count,
    boot_disk_size_gb=500,
    service_account=SERVICE_ACCOUNT,
)

print("LoRA adapter was saved in: ", lora_output_dir)
print("Trained and merged models were saved in: ", merged_model_output_dir)

In [ ]:
# @title Deploy with TGI
# @markdown This section uploads the model to Model Registry and deploys it on the Endpoint. It takes 15 minutes to 1 hour to finish.
# @markdown Please click "Show Code" to see more details.


print("Deploying models in: ", merged_model_output_dir)

# Please finds Vertex AI prediction supported accelerators and regions in [here](https://cloud.google.com/vertex-ai/docs/predictions/configure-compute).
# Sets 1 L4 (24G) to deploy Gemma models.
machine_type = "g2-standard-12"
accelerator_type = "NVIDIA_L4"
accelerator_count = 1

check_quota(project_id=PROJECT_ID,
            region=REGION,
            accelerator_type=accelerator_type,
            accelerator_count=accelerator_count,
            is_for_training=False)

# Note that larger token counts will require more GPU memory.
max_input_length = 512
max_total_tokens = 1024
max_batch_prefill_tokens = 2048

model, endpoint = deploy_model_tgi(
    model_name=get_job_name_with_datetime(prefix="gemma-tgi-serve"),
    model_id=merged_model_output_dir,
    service_account=SERVICE_ACCOUNT,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    max_input_length=max_input_length,
    max_total_tokens=max_total_tokens,
    max_batch_prefill_tokens=max_batch_prefill_tokens,
)
print("endpoint_name:", endpoint.name)

In [ ]:
# @title Predict
# @markdown Once deployment succeeds, you can send requests to the endpoint with text prompts.

# @markdown Here we use an example from the [timdettmers/openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco) to show the finetuning outcome:

# @markdown ```
# @markdown ### Human: How would the Future of AI in 10 Years look?### Assistant: Predicting the future is always a challenging task, but here are some possible ways that AI could evolve over the next 10 years: Continued advancements in deep learning: Deep learning has been one of the main drivers of recent AI breakthroughs, and we can expect continued advancements in this area. This may include improvements to existing algorithms, as well as the development of new architectures that are better suited to specific types of data and tasks. Increased use of AI in healthcare: AI has the potential to revolutionize healthcare, by improving the accuracy of diagnoses, developing new treatments, and personalizing patient care. We can expect to see continued investment in this area, with more healthcare providers and researchers using AI to improve patient outcomes. Greater automation in the workplace: Automation is already transforming many industries, and AI is likely to play an increasingly important role in this process. We can expect to see more jobs being automated, as well as the development of new types of jobs that require a combination of human and machine skills. More natural and intuitive interactions with technology: As AI becomes more advanced, we can expect to see more natural and intuitive ways of interacting with technology. This may include voice and gesture recognition, as well as more sophisticated chatbots and virtual assistants. Increased focus on ethical considerations: As AI becomes more powerful, there will be a growing need to consider its ethical implications. This may include issues such as bias in AI algorithms, the impact of automation on employment, and the use of AI in surveillance and policing. Overall, the future of AI in 10 years is likely to be shaped by a combination of technological advancements, societal changes, and ethical considerations. While there are many exciting possibilities for AI in the future, it will be important to carefully consider its potential impact on society and to work towards ensuring that its benefits are shared fairly and equitably.
# @markdown ```

# @markdown Please click "Show Code" to see more details.

# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint.name` allows us to get the
#   endpoint name of the endpoint `endpoint` created in the cell
#   above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint = aiplatform.Endpoint(aip_endpoint_name)

prompt = "How would the Future of AI in 10 Years look?"  # @param {type: "string"}
max_tokens = 128  # @param {type:"integer"}
temperature = 1.0  # @param {type:"number"}
top_p = 0.9  # @param {type:"number"}
top_k = 1  # @param {type:"integer"}

# Overides max_tokens and top_k parameters during inferences.
# If you encounter the issue like `ServiceUnavailable: 503 Took too long to respond when processing`,
# you can reduce the max length, such as set max_tokens as 20.
instances = [
    {
        "inputs": f"### Human: {prompt}### Assistant: ",
        "parameters": {
            "max_new_tokens": max_tokens,
            "temperature": temperature,
            "top_p": top_p,
            "top_k": top_k,
        },
    },
]
response = endpoint.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

In [ ]:
# @markdown Text moderation analyzes a document against a list of safety attributes, which include "harmful categories" and topics that may be considered sensitive.
# @markdown Please click "Show Code" to see more details.

for generated_text in response.predictions:
    # Send a request to the API.
    response = moderate_text(generated_text)
    # Show the results.
    show_text_moderation(generated_text, response)

## Clean up resources

In [ ]:
# Delete the train job.
train_job.delete()

# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()

# Delete Cloud Storage bucket that was created.
if BUCKET_URI == f"gs://{PROJECT_ID}-tmp-{now}":
    ! gsutil -m rm -r $STAGING_BUCKET